In [2]:
import pandas as pd

In [ ]:
# Cargar el archivo original
df = pd.read_csv("DEGS_data2.csv")

# Separar cada comparación
comparaciones = df["comparacion"].unique()

# Sufijos para renombrar las muestras como espera el pipeline
sufijos = {
    "WT": ["WT_1.gProcessedSignal", "WT_2.gProcessedSignal"],
    "Clon": ["Clon_4.gProcessedSignal", "Clon_10.gProcessedSignal"],
    "RNA7_FN1": ["RNA7_FN1_1.gProcessedSignal", "RNA7_FN1_2.gProcessedSignal"],
    "RNA5_TLR4": ["RNA5_TLR4_1.gProcessedSignal", "RNA5_TLR4_2.gProcessedSignal"]
}

# Inicializamos diccionarios con valores por muestra
expresion = {sample: {} for sublist in sufijos.values() for sample in sublist}

# Agregamos los genes a cada muestra
genes = df["Gene_Symbol"].unique()

# WT es el control, se le asigna 0 a todos los genes
for sample in sufijos["WT"]:
    for gene in genes:
        expresion[sample][gene] = 0

# Para las demás comparaciones usamos los valores de fc
for _, row in df.iterrows():
    gene = row["Gene_Symbol"]
    comp = row["comparacion"]
    fc = row["fc"]
    muestra = comp.split("/")[0]
    if muestra in sufijos:
        for sample in sufijos[muestra]:
            expresion[sample][gene] = fc

In [5]:
# Crear DataFrame final
df_final = pd.DataFrame.from_dict(expresion, orient="index")
df_final.index.name = "Sample"

# Guardar archivo listo para read_dataset
df_final.to_csv("subSVMs.csv")

In [3]:
# Cargar el archivo original
df = pd.read_csv("subSVMs.csv", index_col=0)

# Quitar la columna 'X1' si no es relevante para el modelo
if 'X1' in df.columns:
    df = df.drop(columns='X1')

# Crear subconjuntos
wt_samples = df[df.index.str.startswith("WT_")]

# Función auxiliar para guardar comparaciones
def save_comparison(group_name, prefix):
    group_samples = df[df.index.str.contains(prefix)]
    subset = pd.concat([wt_samples, group_samples])
    subset.to_csv(f"WT_vs_{group_name}.csv")

# Guardar los tres subconjuntos
save_comparison("TLR4", "RNA5_TLR4")
save_comparison("FN1", "RNA7_FN1")
save_comparison("Clon", "Clon_")

print("Archivos guardados: WT_vs_TLR4.csv, WT_vs_FN1.csv, WT_vs_Clon.csv")

Archivos guardados: WT_vs_TLR4.csv, WT_vs_FN1.csv, WT_vs_Clon.csv


In [5]:
# Cargar el archivo Excel
file_path = 'data1-raw-test.xlsx'  # Cambia esta ruta al archivo que estás utilizando
data = pd.read_excel(file_path)

# Extraer las columnas específicas, incluyendo Gene_Symbol
columns_to_extract = ['Gene_Symbol', 'WT_1.gProcessedSignal', 'WT_2.gProcessedSignal', 'RNA5_TLR4_1.gProcessedSignal', 'RNA5_TLR4_2.gProcessedSignal']
extracted_data = data[columns_to_extract]

# Mostrar las primeras filas de los datos extraídos (opcional)
print(extracted_data.head())

# Guardar el archivo con solo las columnas extraídas como CSV
extracted_data.to_csv('WT_TLR4.csv', index=False)


  Gene_Symbol  WT_1.gProcessedSignal  WT_2.gProcessedSignal  \
0         NaN            1385.951000            2462.188000   
1   LINC02615               3.412794               3.409920   
2   LOC145474              20.109310              65.763980   
3       LINP1             125.424300             241.432900   
4    KIAA0040               2.245853               3.463457   

   RNA5_TLR4_1.gProcessedSignal  RNA5_TLR4_2.gProcessedSignal  
0                    984.107400                    777.811100  
1                      6.798984                      7.250548  
2                    101.170800                     77.031590  
3                     66.685530                     40.632620  
4                      7.226670                      3.053636  


## Generar archivos DE


In [10]:
def procesar_y_guardar_degs(archivo_entrada):
    # Leer el archivo de expresión diferencial
    df = pd.read_csv(archivo_entrada)

    # Filtrar solo las filas que tienen un valor para 'comparacion' específico
    comparaciones = df['comparacion'].unique()  # Extraemos todos los contrastes únicos

    # Iteramos sobre cada comparación para crear archivos separados
    for comparacion in comparaciones:
        # Filtrar el dataframe para el contraste actual
        df_comparacion = df[df['comparacion'] == comparacion]

        # Seleccionar las columnas necesarias: Gene_Symbol, fc, pval
        df_comparacion = df_comparacion[['Gene_Symbol', 'fc', 'pval']]

        # Guardar el archivo CSV para este contraste
        output_filename = f"DE_{comparacion.replace('/', '_')}.csv"  # Reemplazamos '/' por '_'
        df_comparacion.to_csv(output_filename, index=False)

        print(f"Archivo generado: {output_filename}")

In [11]:
# Usar la función con el archivo que mencionaste
archivo_entrada = "DEGs_data2.csv"
procesar_y_guardar_degs(archivo_entrada)

Archivo generado: DE_Clon_WT.csv
Archivo generado: DE_RNA7_FN1_WT.csv
Archivo generado: DE_RNA5_TLR4_WT.csv


In [3]:
    
# Ruta del archivo que deseas analizar
archivo = "WT_vs_Clon.csv"  # Reemplaza con la ruta del archivo a analizar

describir_archivo(archivo)

Primeras filas del archivo:
                     Sample  LOC145474     LINP1  LINC01512  LINC02381  \
0     WT_1.gProcessedSignal   0.000000  0.000000   0.000000   0.000000   
1     WT_2.gProcessedSignal   0.000000  0.000000   0.000000   0.000000   
2   Clon_4.gProcessedSignal   1.451998 -1.650409  -1.168084   1.520308   
3  Clon_10.gProcessedSignal   1.451998 -1.650409  -1.168084   1.520308   

   LINC01122  LOC100130691  XLOC_l2_015760  MIR100HG     HIPK2  ...  \
0   0.000000      0.000000         0.00000  0.000000  0.000000  ...   
1   0.000000      0.000000         0.00000  0.000000  0.000000  ...   
2  -1.026289      1.311896         1.01225 -2.026618 -1.028291  ...   
3  -1.026289      1.311896         1.01225 -2.026618 -1.028291  ...   

   RBM26-AS1  LINC00482  lnc-GTF2IRD2-1  PITRM1-AS1  LINC01541     X2299  \
0   0.000000   0.000000        0.000000    0.000000   0.000000  0.000000   
1   0.000000   0.000000        0.000000    0.000000   0.000000  0.000000   
2   1.332849  -1.